2일차 수업의 핵심 내용인 분류 알고리즘(로지스틱 회귀,
의사결정나무)과 모델 평가(정밀도, 재현율, ROC)를 복습하고 실습할
수 있는 7교시 미니 프로젝트를 구성합니다.

주제는 건강검진 데이터를 활용한 ‘심혈관 질환(Cardiovascular Disease)
위험군 진단’입니다. 이 주제는 특성(변수)의 해석이 중요하며, 환자를
놓치지 않는 재현율(Recall)이 강조되는 전형적인 헬스케어 문제입니다.

------------------------------------------------------------------------

# 📘 \[7교시 미니 프로젝트\] 건강검진 데이터를 활용한 심혈관 질환 진단

## 1. 프로젝트 개요 및 데이터 정의

### 1.1. 프로젝트 목표

- 시나리오: 당신은 ’메디컬 AI 센터’의 연구원이다. 건강검진
  수진자들의 기본 정보와 검사 결과를 바탕으로, 향후 심혈관 질환이 발병할
  위험이 높은 고위험군을 조기에 선별(Screening)하는 모델을 개발해야
  한다.
- 학습 목표:
  1.  로지스틱 회귀와 의사결정나무 두 가지 모델을 모두
      학습시키고 비교한다.
  2.  질병 진단에서 가장 중요한 재현율(Recall)을 중심으로 모델을
      평가한다.
  3.  ROC 곡선을 그리고 AUC 점수를 계산하여 모델의 성능을
      입증한다.
  4.  (심화) 변수 중요도를 분석하여 의사에게 진단 근거를 설명한다.

### 1.2. 사용할 데이터셋 (Synthetic Heart Data)

실제 심장병 데이터셋(UCI Heart Disease)의 특성을 모방한 가상 데이터를
생성하여 사용한다.

|     변수명      |             설명              |  타입  |        예시        |
|:----------------:|:----------------:|:----------------:|:----------------:|
|     age     |             나이              | 수치형 |      45, 52 …      |
|     sex     |             성별              | 범주형 |  0: 여성, 1: 남성  |
|     bp      |      수축기 혈압 (mmHg)       | 수치형 |     120, 145 …     |
| cholesterol |      콜레스테롤 (mg/dL)       | 수치형 |     200, 240 …     |
|   smoker    |           흡연 여부           | 범주형 | 0: 비흡연, 1: 흡연 |
|  exercise   |     주당 운동 시간 (시간)     | 수치형 |     0, 2, 5 …      |
|   target    | 심혈관 질환 유무 (Target) | 범주형 |  0: 정상, 1: 질환  |

------------------------------------------------------------------------

## 2. 프로젝트 개발 가이드 (Step-by-Step)

### Step 1. 데이터 생성 및 탐색 (EDA)

- 제공된 코드로 데이터를 생성한다.
- 타겟(`target`)의 비율을 확인한다. (환자와 정상인의 비율이 깨져있는지
  확인)
- 혈압(`bp`)이 높은 그룹에서 질환 발병률이 높은지 시각화해본다.

### Step 2. 데이터 전처리 및 분할

- 스케일링: 로지스틱 회귀는 숫자의 크기에 민감하다. `age`, `bp`,
  `cholesterol` 같은 수치형 변수를 `StandardScaler`로 표준화한다.
- 데이터 분할: `train_test_split`을 사용하여 7:3 비율로 나눈다.
  (`random_state=42`)

### Step 3. 모델링 (Model Comparison)

- 모델 A: `LogisticRegression`을 학습시킨다.
- 모델 B: `DecisionTreeClassifier`를 학습시킨다. (과적합 방지를 위해
  `max_depth`를 3~5 정도로 제한한다.)

### Step 4. 평가 및 해석 (Evaluation)

- 두 모델의 Confusion Matrix를 출력하고 비교한다.
- Classification Report를 통해 Recall(재현율)을 확인한다. 환자를
  놓치지 않는 것이 목표이므로 Recall이 높은 모델이 더 좋다.
- ROC 곡선을 그리고 두 모델의 AUC를 비교하여 최종 챔피언 모델을
  선정한다.

------------------------------------------------------------------------

## 3. 구현 힌트 (Hints)

- 힌트 1 (스케일링 주의): 의사결정나무는 스케일링이 필요 없지만,
  로지스틱 회귀를 위해 스케일링을 수행해도 의사결정나무 성능에는
  악영향이 없다. 편의상 전처리된 데이터를 두 모델 모두에 사용해도 된다.
- 힌트 2 (해석): 로지스틱 회귀에서는 `.coef_` (계수)를 확인하고,
  의사결정나무에서는 `.feature_importances_`를 확인하여 주요 위험 요인을
  찾는다.
- 힌트 3 (ROC): `roc_curve` 함수에는 예측된 클래스(0, 1)가 아니라
  예측 확률(`predict_proba`의 양성 클래스 확률)을 넣어야 한다.

------------------------------------------------------------------------

## 4. 최종 구현 코드 및 결과

### 4.1. 데이터 생성 및 탐색

``` python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

# 1. 가상 데이터 생성
np.random.seed(42)
n = 1000

age = np.random.randint(30, 80, n)
sex = np.random.choice([0, 1], n)  # 0:F, 1:M
bp = np.random.normal(125, 15, n)  # 혈압
chol = np.random.normal(200, 30, n) # 콜레스테롤
smoker = np.random.choice([0, 1], n, p=[0.7, 0.3])
exercise = np.random.randint(0, 7, n) # 운동 시간

# 질병 발생 로직 (Risk Score 계산)
# 나이, 혈압, 콜레스테롤, 흡연은 위험요인(+), 운동은 예방요인(-)
risk_score = -5 + (0.06 * age) + (0.03 * bp) + (0.01 * chol) + (1.5 * smoker) - (0.5 * exercise)
# 시그모이드 함수로 확률 변환
prob = 1 / (1 + np.exp(-risk_score))
# 확률 기반으로 0 또는 1 생성
target = [1 if p > 0.5 else 0 for p in prob]

df = pd.DataFrame({
    'age': age, 'sex': sex, 'bp': bp, 'cholesterol': chol,
    'smoker': smoker, 'exercise': exercise, 'target': target
})

print("데이터 비율 확인:")
print(df['target'].value_counts())

# 시각화: 흡연 여부에 따른 질병 비율
sns.countplot(x='smoker', hue='target', data=df)
plt.title("Heart Disease by Smoking Status")
plt.show()
```

### 4.2. 전처리 (분할 및 스케일링)

``` python
# 1. X, y 분리
X = df.drop('target', axis=1)
y = df['target']

# 2. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. 스케일링 (수치형 변수만 표준화)
# 로지스틱 회귀를 위해 필수
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("전처리 완료.")
```

### 4.3. 모델링 (로지스틱 vs 의사결정나무)

``` python
# 모델 A: 로지스틱 회귀
log_model = LogisticRegression()
log_model.fit(X_train_scaled, y_train)

# 모델 B: 의사결정나무 (깊이 제한 4)
tree_model = DecisionTreeClassifier(max_depth=4, random_state=42)
tree_model.fit(X_train_scaled, y_train) # 트리는 스케일링 필요 없지만, 편의상 사용

# 예측 (확률 포함)
y_pred_log = log_model.predict(X_test_scaled)
y_prob_log = log_model.predict_proba(X_test_scaled)[:, 1]

y_pred_tree = tree_model.predict(X_test_scaled)
y_prob_tree = tree_model.predict_proba(X_test_scaled)[:, 1]

print("두 모델 학습 완료.")
```

### 4.4. 평가 및 결과 비교 (Recall & AUC)

``` python
# 1. Classification Report 비교
print("=== [모델 A] 로지스틱 회귀 성적표 ===")
print(classification_report(y_test, y_pred_log))

print("\n=== [모델 B] 의사결정나무 성적표 ===")
print(classification_report(y_test, y_pred_tree))

# 2. AUC 점수 계산
auc_log = roc_auc_score(y_test, y_prob_log)
auc_tree = roc_auc_score(y_test, y_prob_tree)

print(f"Logistic AUC: {auc_log:.4f}")
print(f"Tree AUC:     {auc_tree:.4f}")

# 3. ROC 곡선 시각화
fpr_log, tpr_log, _ = roc_curve(y_test, y_prob_log)
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_prob_tree)

plt.figure(figsize=(8, 6))
plt.plot(fpr_log, tpr_log, label=f'Logistic (AUC={auc_log:.2f})', color='blue')
plt.plot(fpr_tree, tpr_tree, label=f'Decision Tree (AUC={auc_tree:.2f})', color='green')
plt.plot([0, 1], [0, 1], 'k--') # 기준선
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve Comparison')
plt.legend()
plt.show()

# 4. 변수 중요도 (로지스틱 회귀 기준)
coef_df = pd.DataFrame({'Feature': X.columns, 'Weight': log_model.coef_[0]})
coef_df = coef_df.sort_values(by='Weight', ascending=False)
print("\n=== [분석] 심혈관 질환 주요 위험 인자 ===")
display(coef_df)

# [결과 해석 예시]
# 로지스틱 회귀의 AUC가 0.94로 매우 높게 나타났다. (데이터가 선형적 관계라 유리함)
# 변수 중요도를 보면 smoker(흡연)의 가중치가 가장 높고, exercise(운동)는 음수로 나타난다.
# 즉, "금연과 운동이 심장병 예방의 핵심"이라는 의학적 상식과 일치하는 결과를 얻었다.
```